In [213]:

import numpy as np
import pandas as pd
import sklearn as sk
import csv
import jieba
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

def makeStopWord():
    from os import listdir
    stopword = set()
    for file in listdir('stopwords'):
        if file.endswith("txt"):
            with open(f'stopwords/{file}')as f:
                lines = f.readlines()
                for line in lines:
                    words = jieba.lcut(line,cut_all = False)
                    for word in words:
                        stopword.add(word.strip())
    return stopword

def split(sentence):
    from re import match
    words = jieba.lcut(sentence,HMM = False)
    words = [x.strip() for x in words]
    #不能在停用词中，去除数字
    return list(filter(lambda x: x not in stopwords and match("\d+$",x) == None,words))

def cleanComment(comment):
    import re
    comment = re.sub('#.*?#', '', comment)
    comment = re.sub('//@.*?:', '', comment)
    comment = re.sub('//@.*?：', '', comment)
    comment = re.sub('//.*?:', '', comment)
    comment = re.sub('//.*?：', '', comment)
    comment = re.sub('【.*?】', '', comment)
    comment = re.sub('《.*?》', '', comment)
    comment = re.sub('//.*?//', '', comment)
    comment = re.sub('@.*?：', '', comment)
    comment = re.sub('@.*?:', '', comment)
    comment = re.sub('『.*?』', '', comment)
    comment = re.sub(r'\d', '', comment)
    comment = re.sub(r"(回复)?(//)?\s*@\S*?\s*(:| |$)", " ", comment)  # 去除正文中的@和回复/转发中的用户名
    comment = re.sub(r"\[\S+\]", "", comment)      # 去除表情符号
    comment = re.sub(r"#\S+#", "", comment)      # 保留话题内容
    comment = comment.replace("转发微博", "")       # 去除无意义的词语
    comment = re.sub(r"\s+", " ", comment) # 合并正文中过多的空格
    return comment

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train_data = train_data.fillna('')
test_data = test_data.fillna('')


train_data['merge'] = train_data['content']+train_data['comment_all']
test_data['merge'] = test_data['content']+test_data['comment_all']

stopwords = makeStopWord()

train_data['cut_merge'] = train_data['merge'].apply(cleanComment)
test_data['cut_merge'] = test_data['merge'].apply(cleanComment)

train_data['cut_merge'] = train_data['merge'].apply(split)
test_data['cut_merge'] = test_data['merge'].apply(split)

train_data['cut_merge'].to_csv('train_cut.csv')
test_data['cut_merge'].to_csv('test_cut.csv')


X_train = train_data['cut_merge']
Y_train = train_data['label']

X_test = test_data['cut_merge']



train_data.head(10)




,Unnamed: 0,content,comment_all,label,merge,cut_merge
0,9982,\t\t#王嘉尔##热血街舞团# 王嘉尔或因加盟爱奇艺《热血街舞团》遭优酷全面封杀，目前优酷...,不想理你，大家不要供献热度，谣言止于智者\t优酷和汪涵合作深入，而何炅一直特别挺王嘉尔，这…...,0,\t\t#王嘉尔##热血街舞团# 王嘉尔或因加盟爱奇艺《热血街舞团》遭优酷全面封杀，目前优酷...,"[王, 嘉, 热血, 街, 舞团, 王, 嘉, 加盟, 爱, 奇, 艺, 热血, 街, 舞团..."
1,9983,谁的群最多，帮忙转一下，找一名弥度职中学生，叫刘冻雪，16岁，请速回鲁甸县医院，妈妈伤的很严...,,-1,谁的群最多，帮忙转一下，找一名弥度职中学生，叫刘冻雪，16岁，请速回鲁甸县医院，妈妈伤的很严...,"[群, 帮忙, 转, 找, 一名, 弥, 度, 职, 中学生, 刘, 冻, 雪, 岁, 请,..."
2,9984,\t\t妈蛋，以色列在战术层面的这些神操作真是令人叹为观止 以战机以俄罗斯战机为掩护并释放电...,,0,\t\t妈蛋，以色列在战术层面的这些神操作真是令人叹为观止 以战机以俄罗斯战机为掩护并释放电...,"[妈, 蛋, 以色列, 战术, 层面, 神, 操作, 令人, 叹为观止, 战机, 俄罗斯, ..."
3,9985,帮朋友转的，急找孩子，求转，求帮忙寻人启事13940292999。有线索酬金10万，今天上午...,你受骗上当了，这是虚假消息！请看辟谣网址（O谣场现形记）。尽快删除帖子，因为已被举报。\t回...,-1,帮朋友转的，急找孩子，求转，求帮忙寻人启事13940292999。有线索酬金10万，今天上午...,"[帮, 朋友, 转, 急, 找, 孩子, 求, 转, 求, 帮忙, 寻人启事, 线索, 酬金..."
4,9986,\t\t#不情愿的新娘# 看的吻戏最多的一部泰剧从第一集到第四集数不清哈哈哈哈虽然是沙雕剧情...,后面要虐[泪]\t都在哪里看的?\t,0,\t\t#不情愿的新娘# 看的吻戏最多的一部泰剧从第一集到第四集数不清哈哈哈哈虽然是沙雕剧情...,"[情愿, 新娘, 吻, 戏, 一部, 泰, 剧, 第一集, 第四集, 数不清, 哈哈哈哈, ..."
5,9987,#商界大搜罗#【滴滴司机打投资人张桓！滴滴回应称：抱歉，很震惊，将垫付医疗费】4月28日，滴...,狡猾的炒作！！！！[挖鼻]\t别人是来赚钱的，应该有别的原因吧\t滴滴:很震惊，已交由律师处...,1,#商界大搜罗#【滴滴司机打投资人张桓！滴滴回应称：抱歉，很震惊，将垫付医疗费】4月28日，滴...,"[商界, 搜罗, 滴滴, 司机, 投资人, 张, 桓, 滴滴, 回应, 称, 抱歉, 震惊,..."
6,9988,#吉林市春芽中东幼儿园事件#就怕事不大！,,1,#吉林市春芽中东幼儿园事件#就怕事不大！,"[吉林市, 春, 芽, 中东, 幼儿园, 事件, 怕事]"
7,9989,爸爸买了我很喜欢的芝士蛋糕和三明治……可惜我现在什么味道都吃不出来也闻不到气味……嗅觉味觉完...,,1,爸爸买了我很喜欢的芝士蛋糕和三明治……可惜我现在什么味道都吃不出来也闻不到气味……嗅觉味觉完...,"[爸爸, 买, 喜欢, 芝士, 蛋糕, 三明治, 可惜, 味道, 吃不出来, 闻, 不到, ..."
8,9990,#平安守护#2018年1月21日晚，尉氏县城一白色轿车闯红灯，执勤的尉氏县公安局交警大队民警...,,1,#平安守护#2018年1月21日晚，尉氏县城一白色轿车闯红灯，执勤的尉氏县公安局交警大队民警...,"[平安, 守护, 年, 月, 晚, 尉氏县, 城, 白色, 轿车, 闯红灯, 执勤, 尉氏县..."
9,9991,马天宇，因为是你，所以我相信@馬天宇 对于凉生这部电视剧，虽然不知道到底发生了什么，那些说...,,0,马天宇，因为是你，所以我相信@馬天宇 对于凉生这部电视剧，虽然不知道到底发生了什么，那些说...,"[马, 天宇, 馬, 天宇, 凉, 生, 这部, 电视剧, 发生, 减, 马, 天宇, 戏份..."


In [259]:

Vectorizer = CountVectorizer(max_df = 0.8,
                            min_df = 3,
                            token_pattern = u'(?u)\\b[^\\d\\W]\\w+\\b',
                            stop_words =stopwords )



In [319]:
nb = MultinomialNB()

X_train_vect =  Vectorizer.fit_transform((" ".join(x) for x in train_data['cut_merge'].values))

nb.fit(X_train_vect, Y_train)



MultinomialNB()

In [343]:
X_vec = Vectorizer.transform((" ".join(x) for x in test_data['cut_merge'].values))

nb_result = nb.predict(X_vec)

test_data['result'] = nb_result

np.array(nb_result)

np.savetxt('result.txt',np.array(nb_result),fmt = '%d')



